# Day 14: Extended Polymerization

The incredible pressures at this depth are starting to put a strain on your submarine. The submarine has polymerization equipment that would produce suitable materials to reinforce the submarine, and the nearby volcanically-active caves should even have the necessary input elements in sufficient quantities.

The submarine manual contains instructions for finding the optimal polymer formula; specifically, it offers a polymer template and a list of pair insertion rules (your puzzle input). You just need to work out what polymer would result after repeating the pair insertion process a few times.

For example:

```text
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C
```

The first line is the polymer template - this is the starting point of the process.

The following section defines the pair insertion rules. A rule like `AB -> C` means that when elements `A` and `B` are immediately adjacent, element `C` should be inserted between them. These insertions all happen simultaneously.

So, starting with the polymer template `NNCB`, the first step simultaneously considers all three pairs:

    The first pair (NN) matches the rule NN -> C, so element C is inserted between the first N and the second N.
    The second pair (NC) matches the rule NC -> B, so element B is inserted between the N and the C.
    The third pair (CB) matches the rule CB -> H, so element H is inserted between the C and the B.

Note that these pairs overlap: the second element of one pair is the first element of the next pair. Also, because all pairs are considered simultaneously, inserted elements are not considered to be part of a pair until the next step.

After the first step of this process, the polymer becomes `NCNBCHB`.

Here are the results of a few steps using the above rules:

```text
Template:     NNCB
After step 1: NCNBCHB
After step 2: NBCCNBBBCBHCB
After step 3: NBBBCNCCNBBNBNBBCHBHHBCHB
After step 4: NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB
```

This polymer grows quickly. After step 5, it has length 97; After step 10, it has length 3073. After step 10, `B` occurs 1749 times, `C` occurs 298 times, `H` occurs 161 times, and `N` occurs 865 times; taking the quantity of the most common element (`B`, 1749) and subtracting the quantity of the least common element (`H`, 161) produces 1749 - 161 = 1588.

Apply 10 steps of pair insertion to the polymer template and find the most and least common elements in the result. **What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?**

In [1]:
# Python imports
from collections import Counter, defaultdict
from pathlib import Path
from typing import Callable, Dict, Generator, Iterable, List, Set, Tuple

# Paths to data
testpath = Path("day14_test.txt")
datapath = Path("day14_data.txt")

We load the data in as a starting polymer string, and a dictionary set of rules associating a symbol pair with the first two characters of its three-character replacement. For example, `AB -> C` would have the dictionary item `AB: AC`.

In [2]:
def load_input(fpath: Path) -> Tuple[str, Dict]:
    """Return starting polymer and rules
    
    :param fpath:  Path to data file
    """
    state = "polymer"  # state for parser
    rules = {}  # holds parsed rules
    
    with fpath.open("r") as ifh:
        for line in [_.strip() for _ in ifh.readlines()]:
            if len(line) == 0:  # switch parser state
                state = "rules"
            elif state == "polymer":  # read in polymer string
                polymer = line.strip()
            elif state == "rules":  # read in rules as dictionary
                key, val = line.strip().split(" -> ")
                rules[key] = val
                
    return polymer, rules

For the first part of the puzzle, we write two functions. One expands the polymer at a single step, by 

In [3]:
def expand_polymer(polymer: str, rules: Dict) -> str:
    """Expand the polymer, using the passed rules
    
    :param polymer:  current polymer string
    :param rules:  rules for inserting monomers in a pair
    """
    expanding = []  # hold list of new polymer elements
    
    # Split polymer into consecutive pairs, and apply rules to each pair
    polysplit = [polymer[idx:idx+2] for idx in range(len(polymer) - 1)]
    for pair in polysplit:
        expanding.append(f"{pair[0]}{rules[pair]}")
            
    # concatenate polymer elements, and add last character from input
    return "".join(expanding) + polymer[-1]

def polymer_score(polymer: str) -> int:
    """Return difference between counts of most and least abundant monomer
    
    :param polymer:  polymer string
    """
    # Count occurrences of each monomer, and sort by abundance
    counts = sorted([(v, k) for k, v in Counter(list(polymer)).items()])
    
    # Return difference of most and least abundant counts
    return counts[-1][0] - counts[0][0]

Applying these to the test data:

In [4]:
polymer, rules = load_input(testpath)
for idx in range(10):
    polymer = expand_polymer(polymer, rules)
    print(f"{idx + 1}: {len(polymer)=}, {polymer_score(polymer)=}")

1: len(polymer)=7, polymer_score(polymer)=1
2: len(polymer)=13, polymer_score(polymer)=5
3: len(polymer)=25, polymer_score(polymer)=7
4: len(polymer)=49, polymer_score(polymer)=18
5: len(polymer)=97, polymer_score(polymer)=33
6: len(polymer)=193, polymer_score(polymer)=82
7: len(polymer)=385, polymer_score(polymer)=160
8: len(polymer)=769, polymer_score(polymer)=366
9: len(polymer)=1537, polymer_score(polymer)=727
10: len(polymer)=3073, polymer_score(polymer)=1588


And to the puzzle data

In [5]:
polymer, rules = load_input(datapath)
for idx in range(10):
    polymer = expand_polymer(polymer, rules)
print(f"{idx + 1}: {len(polymer)=} - {polymer_score(polymer)=}")

10: len(polymer)=19457 - polymer_score(polymer)=2915


## Puzzle 2

The resulting polymer isn't nearly strong enough to reinforce the submarine. You'll need to run more steps of the pair insertion process; a total of 40 steps should do it.

In the above example, the most common element is `B` (occurring 2192039569602 times) and the least common element is `H` (occurring 3849876073 times); subtracting these produces 2188189693529.

Apply 40 steps of pair insertion to the polymer template and find the most and least common elements in the result. What do you get if you take the quantity of the most common element and subtract the quantity of the least common element?

My solution to part 1 does not scale well, so a new approach is required.

We note that each symbol pair in the polymer produces two new symbol pairs, e.g. the rule `AB -> C` produces the pairs `AC` and `CB`. So, we can keep a count of the frequency of each pair in a dictionary, and update this at each cycle. Then, after the final iteration, we can count the number of times each symbol appears in all the pairs. This will be twice the frequency of that symbol in the polymer string _except for_ the starting and ending polymer symbol, which are only counted once (so we increment these). The answer will then be half the difference between the most and least common symbol counts.

In [6]:
def count_pair_freqs(polymer: str) -> Counter:
    """Returns Counter of pair frequencies in a polymer sequence
    
    :param polymer:  polymer string
    """
    return Counter([polymer[idx:idx+2] for idx in range(len(polymer) - 1)])

def make_pair_rules(rules: Dict) -> Dict:
    """Returns new rule dictionary associating one pair with two output pairs
    
    :param rules:  rule dictionary associating one pair with a monomer insert
    """
    return {key: (key[0] + val, val + key[1]) for key, val in rules.items()}

def update_pairs(freqs: Dict, rules: Dict) -> Dict:
    """Returns updated frequency dictionary for each pair on applying rules
    
    :param freqs:  dictionary of symbol pair frequencies
    :param rules:  dictionary of symbol pair updates
    """
    newfreqs = defaultdict(int)  # holds new frequency dictionary
    
    # Update new frequency dictionary with new pair symbols
    for pair in freqs:
        for output in rules[pair]:
            newfreqs[output] += freqs[pair]
            
    return newfreqs

def score_freqs(polymer: str, freqs: Dict) -> int:
    """Return difference in largest and smallest symbol count
    
    :param polymer:  initial polymer string (not current polymer)
    :param freqs:  pair frequency dictionary for current polymer
    """
    letter_counts = defaultdict(int)  # holds counts of individual symbols
    
    # Update letter counts from frequency dictionary
    for key, val in freqs.items():
        letter_counts[key[0]] += val
        letter_counts[key[1]] += val

    # Update/correct counts of start/end symbols in polymer, and sort
    letter_counts[polymer[0]] += 1
    letter_counts[polymer[-1]] += 1
    letter_counts = sorted([(v, k) for k, v in letter_counts.items()])

    return int(0.5 * (letter_counts[-1][0] - letter_counts[0][0]))

Applying this process to the test data:

In [7]:
polymer, rules = load_input(testpath)
pair_freqs = count_pair_freqs(polymer)
pair_rules = make_pair_rules(rules)
for idx in range(40):
    pair_freqs = update_pairs(pair_freqs, pair_rules)
print(idx + 1, score_freqs(polymer, pair_freqs))

40 2188189693529


And to the puzzle data:

In [8]:
polymer, rules = load_input(datapath)
pair_freqs = count_pair_freqs(polymer)
pair_rules = make_pair_rules(rules)
for idx in range(40):
    pair_freqs = update_pairs(pair_freqs, pair_rules)
print(idx + 1, score_freqs(polymer, pair_freqs))

40 3353146900153
